In [9]:
### Library Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf 


In [10]:
### Load in Stock Data
tickers = ['GOOG', 'AAPL']
start_date = '2018-01-01'

def import_stock_data(tickers, start_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date)['Adj Close']
    # Reset index to make headings in the same row
    data.reset_index(inplace = True)

    data['Date'] = pd.to_datetime(data['Date'])

    return data

stock_data = import_stock_data(tickers, start_date)
stock_data


[*********************100%%**********************]  2 of 2 completed


,Date,GOOG,AAPL
0,2018-01-02,40.670975,53.250000
1,2018-01-03,40.663891,54.124001
2,2018-01-04,40.852772,54.320000
3,2018-01-05,41.317902,55.111500
4,2018-01-08,41.164433,55.347000
...,...,...,...
1579,2024-04-12,176.550003,159.190002
1580,2024-04-15,172.690002,156.330002
1581,2024-04-16,169.380005,156.000000
1582,2024-04-17,168.000000,156.880005


In [15]:
### MACD Function for Multiple Stocks
def get_macd(tickers, df):
    # Initialize empty DataFrame to store MACD data
    macd_df = pd.DataFrame(index=df.index)
    macd_vals = {}

    # Loop for MACD calculation and df population
    for t in tickers:
        # Calculate 12-day Exponential Moving Averages (EMA)
        ema_12 = df[t].ewm(span=12, adjust=False).mean()
        # Calculate 26-day Exponential Moving Averages (EMA)
        ema_26 = df[t].ewm(span=26, adjust=False).mean()
        # Calculate MACD line (12-day EMA minus 26-day EMA)
        macd = ema_12 - ema_26
        # Add MACD column to the DataFrame
        macd_df[f'MACD_{t}'] = macd
        # Append last MACD value to rsi_vals dictionary
        macd_vals[t] = macd_df[f'MACD_{t}'].iloc[-1]

    return macd_df, macd_vals

macd_df, macd_vals = get_macd(tickers, stock_data)
print(macd_df)
print("MACD values list: ", macd_vals)

      MACD_GOOG  MACD_AAPL
0      0.000000   0.000000
1     -0.000565   0.069721
2      0.014066   0.139186
3      0.062473   0.255164
4      0.087445   0.361908
...         ...        ...
1579  -1.234146   3.995633
1580  -1.058293   3.745014
1581  -1.172502   3.479657
1582  -1.358706   3.302301
1583  -1.565690   3.171982

[1584 rows x 2 columns]
MACD values list:  {'GOOG': -1.5656896620644432, 'AAPL': 3.171982242640439}
